<a href="https://colab.research.google.com/github/genki-mii/BookShelf/blob/master/hall_scrape_ilandakihabara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

根本的にやり方を変える。
今まではread_htmlで取得して追加でほしい情報も入れていたが、今度はシンプルに欲しい情報だけsoupで取得する

In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install requests-html
!pip3 install pyautogui

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic

In [10]:
import cv2
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
import urllib.request, urllib.error
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import glob

# htmlを取得するところまで
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)

In [3]:
#Googledriveをマウントする
from google.colab import drive
drive.mount('./gdrive')

Mounted at ./gdrive


In [4]:
!mkdir images

##台データを全て取得する


###店ごとに変更しなければならないところ
1.   12行目の台番号のリスト作成（デフォルトは4と9を抜いている）
2.   19行目のURLリストの日付を指定
3.   111行目の画像の保存先のパス
4.   121行目の画像の読み取り先のパス

###データサイトごとに変更しなければならないところ
1.   133行目のスタートの座標
2.   146行目のy座標1に付き、何PXか

In [13]:
#各リストの作成
url_number_list=[]
no_list=[]
name_list=[]
bb_list=[]
rb_list=[]
art_list=[]
game_list=[]
hall_name_list=[]

#4と9を除いた台番号リストを作成
for i in range(600,1281,1):
  str_i = str(i)
  if '4' in str_i or '9' in str_i:
      continue
  url_number_list.append(i)

#日付は前日の日付を選択する
url_list=["https://papimo.jp/h/00031715/hit/view/{0}/20201122".format(number) for number in url_number_list]

#データの取得
for url in tqdm(url_list):
  time.sleep(1)
  driver.get(url)
  html = driver.page_source.encode('utf-8')
  soup = BeautifulSoup(html, "html.parser")

  #台番の取得
  unitno=soup.find_all("p",attrs={"class":"unit_no"})

  for n in unitno:
    no_list.append(n.string)
  
  #取得したデータから番台を除く
  no_list=[t.replace("番台","") for t in no_list]
  #整数型に置き換える
  no_list1 = list(map(int, no_list))

  #機種名のリスト
  name=soup.find_all("p",attrs={"class":"name"})

  for s in name:
    name_list.append(s.string)

  #BB回数
  bbs=soup.find("tbody").find_all("td")[1]

  for b in bbs:
    bb_list.append(b.string)

  #RB回数
  rbs=soup.find("tbody").find_all("td")[2]

  for r in rbs:
    rb_list.append(r.string)

  #ART回数
  artes=soup.find("tbody").find_all("td")[3]

  for a in artes:
    art_list.append(a.string)

  #総ゲーム数
  game=soup.find("tbody").find_all("td")[6]

  for g in game:
    game_list.append(g.string)
  #ゲーム数だけfloat型に変換。機械割の計算ができないから
  game_list=[t.replace(",","") for t in game_list]
  game_list1 = list(map(float, game_list))

  #ホール名
  hall=soup.find_all("div",attrs={"class":"store-ttl"})

  for u in hall:
    hall_name_list.append(u.string)
  
  time.sleep(1)

#データフレームに変換
df_test=pd.DataFrame({"hall_name":hall_name_list,"unit_number":no_list1,"unit_name":name_list,"bb":bb_list,"rb":rb_list,"art":art_list,"number_game":game_list1})
#オブジェクト型から整数型に変換する
df_data=df_test.astype({"bb": int, "rb": int,"art":int})

　#画像のダウンロード関数
def download_image(url, file_path):
  r = requests.get(url, stream=True)
  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)

#画像URLのリスト
img_url=[]

#画像URLのリストの作成
for url in tqdm(url_list):
  time.sleep(1)
  driver.get(url)
  html = driver.page_source.encode('utf-8')

  import requests as req
  soup = BeautifulSoup(html, "html.parser")

  #グラフの要素を指定する
  img_tags=soup.find("table",attrs={"class":"graph-some"}).find_all("img")[-1]
  img_tag=img_tags.get("src")
  img_url.append(img_tag)
  time.sleep(1)

#画像の保存先
google_drive_save_dir ="/content/gdrive/MyDrive/Colab Notebooks/iland_img"

#img_urlに順番にアクセスして画像をダウンロードする
for index,i_url in enumerate(img_url):
  file_name = "{}.png".format(index)
  image_path = os.path.join(google_drive_save_dir, file_name)
  download_image(url=i_url, file_path=image_path)

#画像データの読み取り先の指定
start = time.time()
file_folders = glob.glob('/content/gdrive/MyDrive/Colab Notebooks/iland_img/*')

#座標のリストを作成
winning_amount1=[]

#画像データの読み取りと座標取得
for read in file_folders:
  # グラフ画像読み込み
  img = cv2.imread(read)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  # グラフ開始地点ピクセルのRGB値を取得（img[y座標][x座標]）
  #サイトごとに指定する必要あり
  color_pattern = img[305][20]
  # 2値化(0 or 255)
  mask = cv2.inRange(img, color_pattern, color_pattern)
  # グラフ部分のみの座標取得
  coordinate = np.stack(np.where(mask == 255), axis=1)
  # x軸の最大値（グラフの終点）
  max_x_axis = coordinate[:, 1].max()
  # x軸の最大値からy座標を求める
  y_axis_target = coordinate[np.where(coordinate[:, 1] == max_x_axis)]
  y_axis = y_axis_target[0][0]
  winning_amount1.append(y_axis)

#差枚数データの調整
px1_data=305
px2_data=6000/145

#差枚数データのリスト
winning_amount=[]

#座標とピクセルデータを元に差枚数を取得する
for w in winning_amount1:
  px3_data=px1_data-w
  px_all=px3_data*px2_data
  winning_amount.append(px_all)

#差枚数データをデータフレームに挿入
df_data["winning_amount"]=winning_amount

#機械割を算出する計算
def kikaiwari(x):
  return ((x.iloc[6]*3)+x.iloc[7]) / (x.iloc[6]*3)
#機械割データをデータフレームに挿入
df_data["expected_value"] = df_data.apply(kikaiwari, axis=1)
#台番号をインデックスに指定する
df_data=df_data.set_index('unit_number')

In [15]:
df_data.to_csv("iland20201122.csv",encoding="utf-8")